In [1]:
import pandas as pd
import json
import math
import datetime
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
#  load JSON file which is not propoer jSON format

In [3]:
records = []
for line in open('C61A.json', 'r'):
    records.append(json.loads(line))

In [4]:
df = pd.json_normalize(records[1:])

In [5]:
df_filter = df[['x.$numberDouble', 'y.$numberDouble', 'z.$numberDouble',
       'timestamp.$numberLong']]

In [6]:
df_filter = df_filter[['x.$numberDouble', 'y.$numberDouble', 'z.$numberDouble']].astype('float32')

In [7]:
df_filter['time'] = df['timestamp.$numberLong'].astype('int64')

In [8]:
df_filter = df_filter.rename(columns= {'x.$numberDouble':'x', 'y.$numberDouble':'y', 'z.$numberDouble':'z'})

In [9]:
# take the minimum and maximum limit of our timeseries
maxs = df_filter['time'].max()

In [37]:
mins = df_filter['time'].min()

In [39]:
mins

1598152118000

In [41]:
maxs - mins

2641789

In [11]:
# transform the min and max value to nearest 100th milliseconds 
minl = int(math.floor(mins / 100.0)) * 100

In [12]:
maxl = int(math.floor(maxs / 100.0)) * 100

In [13]:
#augment the dataframe from minimum to maximum , that will be available for merging with orignial
# data later to fill the values in Augmented data
df_aug = pd.DataFrame({'time':np.arange(minl,maxl+100,100)}, dtype='int64')

In [14]:
#fill values of augmented data with NaN values
df_aug[['x','y','z']] = np.nan

In [16]:
df_aug

,time,x,y,z
0,1598152118000,NaN,NaN,NaN
1,1598152118100,NaN,NaN,NaN
2,1598152118200,NaN,NaN,NaN
3,1598152118300,NaN,NaN,NaN
4,1598152118400,NaN,NaN,NaN
...,...,...,...,...
26413,1598154759300,NaN,NaN,NaN
26414,1598154759400,NaN,NaN,NaN
26415,1598154759500,NaN,NaN,NaN
26416,1598154759600,NaN,NaN,NaN


In [17]:
# apply math.floor here as we did with min and max value of time above
df_filter['time'] = df_filter['time'].apply(lambda x: int(math.floor(x  / 100.0)) * 100)

In [18]:
df = pd.concat([df_filter,df_aug], ignore_index= True)

In [19]:
df.sort_values(by = 'time', ascending = True, inplace = True)

In [20]:
for i in range(len(df)):
    if pd.isna(df.iloc[i, 1]):
        moving = df.iloc[i-5:i, 1].mean()
        df.iloc[i, 1] = moving

In [21]:
for i in range(len(df)):
    if pd.isna(df.iloc[i, 0]):
        moving = df.iloc[i-5:i, 0].mean()
        df.iloc[i, 0] = moving

In [22]:
for i in range(len(df)):
    if pd.isna(df.iloc[i, 2]):
        moving = df.iloc[i-5:i, 2].mean()
        df.iloc[i, 2] = moving

In [23]:
df = df.fillna(method='ffill')

In [24]:
df = df.fillna(method='bfill')

In [25]:
df = df.drop_duplicates(subset=['time'], keep = 'first')

In [26]:
df = df.reset_index(drop= True)

In [27]:
df.to_json('C61A_moving.json', orient='records', lines=True)

In [28]:
df.describe()

,x,y,z,time
count,26418.000000,26418.000000,26418.000000,2.641800e+04
mean,9.875456,-2.818172,0.378598,1.598153e+12
std,8.324475,6.098870,0.776294,7.626364e+05
min,-22.365820,-20.948277,-7.000027,1.598152e+12
25%,4.410391,-7.463614,-0.007840,1.598153e+12
50%,8.965244,-4.782604,0.421211,1.598153e+12
75%,16.616992,0.851050,0.795534,1.598154e+12
max,28.212608,15.736776,7.780260,1.598155e+12


In [29]:
df_filter.describe()

,x,y,z,time
count,6324.000000,6324.000000,6324.000000,6.324000e+03
mean,8.620917,-2.874988,0.448537,1.598153e+12
std,8.862118,6.200922,0.916892,5.989245e+05
min,-22.365820,-21.156071,-7.039297,1.598152e+12
25%,2.354473,-7.952334,0.001258,1.598153e+12
50%,8.588475,-4.664604,0.357588,1.598153e+12
75%,16.200278,1.687044,0.823331,1.598153e+12
max,28.212608,17.321341,7.780260,1.598155e+12


In [36]:
df.head()

,x,y,z,time
0,-3.841946,7.926754,-0.033847,1598152118000
1,-3.447886,8.039709,-0.068640,1598152118100
2,-3.447886,8.039709,-0.068640,1598152118200
3,-3.644916,7.983232,-0.051244,1598152118300
4,-2.924334,7.900291,0.162582,1598152118400


In [31]:
# import matplotlib.pyplot as plt 

In [32]:
# cols = ['x', 'y', 'z', 'time']
# from sklearn.preprocessing import StandardScaler
# stdsc = StandardScaler()
# X_std = stdsc.fit_transform(df[cols].iloc[:,range(0,3)].values
# cov_mat =np.cov(X_std.T)

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26418 entries, 0 to 26417
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x       26418 non-null  float32
 1   y       26418 non-null  float32
 2   z       26418 non-null  float32
 3   time    26418 non-null  int64  
dtypes: float32(3), int64(1)
memory usage: 516.1 KB


In [34]:
df_filter.to_json('C61A_raw.json', orient='records', lines=True)